# Creating xml file for EQRM

## Generated one xml file for EQRM

In [1]:
%matplotlib inline

In [50]:
import os, sys
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import seaborn
seaborn.set_context("poster")

sys.path.insert(0, '/Users/hyeuk/Projects/rmtk/rmtk')
from parsers import vulnerability_model_converter
from vulnerability.common import utils

sys.path.insert(0, '/Users/hyeuk/Projects/scenario_Sydney/code')
from summary_3_2 import compute_vulnerability, compute_vulnerability_retrofit



In [37]:
def write_vul_xml(vuln, flag):

    csv_hdr = 'vuln_set_id,asset_cat,loss_cat,vuln_func_id,distr,imt,iml,mean_lr,stddev_lr'
    output_vul_csv = os.path.join(output_dir, 'vul_curve_{}.csv'.format(flag))
    output_vul_xml = os.path.join(output_dir, 'vul_curve_{}.xml'.format(flag))

    with open(output_vul_csv, 'wb') as f:
        f.write(csv_hdr +'\n')

    for bldg in bldg_classes:
        summary_vul = np.hstack((mmi_range[:, np.newaxis], vuln[bldg][:, np.newaxis], cov_))
        tmp_str = ','.join((vuln_set, asset_cat, loss_cat, bldg, distr, imt))

        with open(output_vul_csv, 'ab') as f:
            for line in summary_vul:
                f.write(tmp_str +',' + ','.join([str(item) for item in line])+'\n')

    # convert csv to xml using RMTK tool
    vulnerability_model_converter.csv_to_xml(output_vul_csv, output_vul_xml)



Original vulnerability curves

In [52]:
mmi_range = np.arange(2.0, 10.05, 0.05)
output_dir = '/Users/hyeuk/Projects/scenario_Sydney/input'


cov_ = np.zeros_like(mmi_range)[:, np.newaxis]
bldg_classes= ['Timber_Pre1945', 'Timber_Post1945', 'URM_Pre1945', 'URM_Post1945']

#vuln_set_id,asset_cat,loss_cat,vuln_func_id,distr,imt,iml,mean_lr,stddev_lr
vuln_set = 'Modfied_Tariq_2016'
asset_cat = 'buildings'
loss_cat = 'economic loss'
distr = 'LN' #
imt = 'MMI'


In [38]:
# original
vul = dict()
for bldg in bldg_classes:
    tmp = []
    for val in mmi_range:
        tmp.append(compute_vulnerability(val, bldg))
    vul[bldg] = np.array(tmp)


In [39]:
write_vul_xml(vul, 'current')

Retrofitted 

In [25]:
# retrofitted
retrofitted = dict()
for bldg in bldg_classes:
    tmp = []
    for val in mmi_range:
        tmp.append(compute_vulnerability_retrofit(val, bldg))
    retrofitted[bldg] = np.array(tmp)


In [40]:
write_vul_xml(retrofitted, 'retrofit')


In [58]:
from lxml import etree

In [83]:
xml_file = os.path.join(output_dir, 'vul_curve_current.xml')

xmlNRML='{http://openquake.org/xmlns/nrml/0.4}'
xmlGML = '{http://www.opengis.net/gml}'

def parseVulnerabilitySet(element):
    taxonomies, iml = [], []
    for subElement in element.iter():
        #print subElement
        if subElement.tag == '%sdiscreteVulnerability' % xmlNRML:
            taxonomies.append(subElement.attrib.get('vulnerabilityFunctionID'))
            #loss_Ratio.append(subElement.attrib.get('lossRatio'))
        elif subElement.tag == '%sIML' % xmlNRML:
            print subElement.text
            #print '{} !'.format(subElement)
        else:
            continue

    #print iml            
    return taxonomies

taxonomies = []
for _, element in etree.iterparse(xml_file):
    if element.tag == '%sdiscreteVulnerabilitySet' % xmlNRML:
        parseVulnerabilitySet(element)
        print subTaxonomies
        for subTaxonomy in subTaxonomies:
            taxonomies.append(subTaxonomy)
        else:
            continue


2.0 2.05 2.1 2.15 2.2 2.25 2.3 2.35 2.4 2.45 2.5 2.55 2.6 2.65 2.7 2.75 2.8 2.85 2.9 2.95 3.0 3.05 3.1 3.15 3.2 3.25 3.3 3.35 3.4 3.45 3.5 3.55 3.6 3.65 3.7 3.75 3.8 3.85 3.9 3.95 4.0 4.05 4.1 4.15 4.2 4.25 4.3 4.35 4.4 4.45 4.5 4.55 4.6 4.65 4.7 4.75 4.8 4.85 4.9 4.95 5.0 5.05 5.1 5.15 5.2 5.25 5.3 5.35 5.4 5.45 5.5 5.55 5.6 5.65 5.7 5.75 5.8 5.85 5.9 5.95 6.0 6.05 6.1 6.15 6.2 6.25 6.3 6.35 6.4 6.45 6.5 6.55 6.6 6.65 6.7 6.75 6.8 6.85 6.9 6.95 7.0 7.05 7.1 7.15 7.2 7.25 7.3 7.35 7.4 7.45 7.5 7.55 7.6 7.65 7.7 7.75 7.8 7.85 7.9 7.95 8.0 8.05 8.1 8.15 8.2 8.25 8.3 8.35 8.4 8.45 8.5 8.55 8.6 8.65 8.7 8.75 8.8 8.85 8.9 8.95 9.0 9.05 9.1 9.15 9.2 9.25 9.3 9.35 9.4 9.45 9.5 9.55 9.6 9.65 9.7 9.75 9.8 9.85 9.9 9.95 10.0
['Timber_Post1945', 'Timber_Pre1945', 'URM_Post1945', 'URM_Pre1945']


In [54]:
doc.iter

<function iter>

In [55]:
doc.iter()